## **Data Augmentation**

- object class inbalance 
- obejct segmentation -> crop and paste 기법으로 데이터 증강 

### **Crop and Paste**

In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

In [ ]:
def resize_and_rotate(cropped, mask, scale=0.95, angle_range=(-30, 30)):
    # 1. Resize
    scale = 0.6 #random.uniform(*scale_range)
    resized_h = max(1, int(cropped.shape[0] * scale))
    resized_w = max(1, int(cropped.shape[1] * scale))
    cropped_resized = cv2.resize(cropped, (resized_w, resized_h), interpolation=cv2.INTER_LINEAR)
    mask_resized = cv2.resize(mask, (resized_w, resized_h), interpolation=cv2.INTER_NEAREST)
    
    # 2. Rotate
    angle = random.uniform(*angle_range)
    center = (resized_w // 2, resized_h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 경계가 튀지 않게 크기 확장
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((resized_h * sin) + (resized_w * cos))
    new_h = int((resized_h * cos) + (resized_w * sin))
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]

    rotated_cropped = cv2.warpAffine(cropped_resized, M, (new_w, new_h), flags=cv2.INTER_LINEAR, borderValue=(0,0,0))
    rotated_mask = cv2.warpAffine(mask_resized, M, (new_w, new_h), flags=cv2.INTER_NEAREST, borderValue=0)
    
    return rotated_cropped, rotated_mask, scale, angle, (new_w, new_h) 


In [ ]:
def transform_polygon(polygon, scale, angle, center, offset):
    # 1. Scale
    poly = polygon * scale
    
    # 2. Rotate
    radians = np.deg2rad(angle)
    rotation_matrix = np.array([
        [np.cos(radians), -np.sin(radians)],
        [np.sin(radians), np.cos(radians)]
    ])
    
    # 중심 기준 이동 후 회전
    poly = poly - center
    poly = np.dot(poly, rotation_matrix.T)
    poly = poly + np.array([center[0], center[1]])
    
    # 3. 최종 위치로 이동
    poly = poly + np.array(offset)
    
    return poly.astype(np.int32)


In [ ]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

# 1. XML에서 polygon 읽기
def load_polygon_from_cvat_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_info = root.find('image')
    width = int(image_info.attrib['width'])
    height = int(image_info.attrib['height'])

    polygons = []
    for polygon in image_info.findall('polygon'):
        points_str = polygon.attrib['points']
        points = []
        for point in points_str.split(';'):
            if point.strip() == '':
                continue
            x, y = map(float, point.split(','))
            points.append([int(x), int(y)])
        polygons.append(np.array(points, dtype=np.int32))

    return polygons, (width, height)

# 2. polygon 영역 crop
def crop_object(image, polygon):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [polygon], 255)
    cropped = cv2.bitwise_and(image, image, mask=mask)
    
    # bounding box 기준으로 잘라내기
    x, y, w, h = cv2.boundingRect(polygon)
    cropped = cropped[y:y+h, x:x+w]
    mask = mask[y:y+h, x:x+w]
    return cropped, mask, (x, y)

# 3. 객체를 paste
def paste_object(base_image, cropped_obj, mask_obj, position):
    x, y = position
    h, w = cropped_obj.shape[:2]
    
    # 영역 체크 (넘치면 자름)
    x = min(x, base_image.shape[1] - w)
    y = min(y, base_image.shape[0] - h)
    
    roi = base_image[y:y+h, x:x+w]

    # Mask 기준으로 복사
    mask_inv = cv2.bitwise_not(mask_obj)
    bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
    fg = cv2.bitwise_and(cropped_obj, cropped_obj, mask=mask_obj)
    
    dst = cv2.add(bg, fg)
    base_image[y:y+h, x:x+w] = dst
    return (x, y)

# 4. polygon 좌표 이동
def move_polygon(polygon, offset):
    offset_x, offset_y = offset
    moved = polygon + np.array([offset_x, offset_y])
    return moved

# ------------------------
# 사용 예시
# ------------------------

# 파일 경로
image_path = './DC/dc1.jpg'
xml_path = r'F:\2.프로젝트\파나시아\2025\model3\DC\dc1.xml'

copy_image_path = r'F:\2.프로젝트\파나시아\2025\model3\dataset\dc 추가하기\4\cam33_2541000.jpg'
# 이미지 로드
image = cv2.imread(image_path)
copy_image = cv2.imread(copy_image_path)


# polygon 로드
polygons, (w, h) = load_polygon_from_cvat_xml(xml_path)

# 결과 이미지 복사본
aug_image = copy_image.copy()
new_polygons = []


for polygon in polygons:
    # crop
    cropped_obj, mask_obj, (crop_x, crop_y) = crop_object(image, polygon)
    
    # resize + rotate
    cropped_obj, mask_obj, scale, angle, (new_w, new_h) = resize_and_rotate(cropped_obj, mask_obj)
    
    # random 위치 생성
    # rand_x = random.randint(0, w - new_w)
    # rand_y = random.randint(0, h - new_h)
    
    rand_x = 1500
    rand_y = 600

    # paste
    paste_offset = paste_object(aug_image, cropped_obj, mask_obj, (rand_x, rand_y))
    
    # polygon 좌표 이동
    center = (cropped_obj.shape[1] // 2, cropped_obj.shape[0] // 2)  # (w//2, h//2)
    moved_poly = transform_polygon(polygon - [crop_x, crop_y], scale, angle, center, paste_offset)
    
    new_polygons.append(moved_poly)
    

# 결과 확인
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

# (선택) 결과 mask 또는 새 XML 저장도 가능


In [ ]:

def save_augmented_image(polygon_image, copy_image_path, xml_path,save_path ):

    # 이미지 로드
    image = cv2.imread(polygon_image)
    copy_image = cv2.imread(copy_image_path)


    # polygon 로드
    polygons, (w, h) = load_polygon_from_cvat_xml(xml_path)

    # 결과 이미지 복사본
    aug_image = copy_image.copy()
    new_polygons = []


    for polygon in polygons:
        # crop
        cropped_obj, mask_obj, (crop_x, crop_y) = crop_object(image, polygon)
        
        # resize + rotate
        cropped_obj, mask_obj, scale, angle, (new_w, new_h) = resize_and_rotate(cropped_obj, mask_obj)
        
        # random 위치 생성
        # rand_x = random.randint(0, w - new_w)
        # rand_y = random.randint(0, h - new_h)

        rand_x = 1500
        rand_y = 600

        # paste
        paste_offset = paste_object(aug_image, cropped_obj, mask_obj, (rand_x, rand_y))
        
        # polygon 좌표 이동
        center = (cropped_obj.shape[1] // 2, cropped_obj.shape[0] // 2)  # (w//2, h//2)
        moved_poly = transform_polygon(polygon - [crop_x, crop_y], scale, angle, center, paste_offset)
        
        new_polygons.append(moved_poly)
        

        # # 결과 확인
        # plt.figure(figsize=(10,10))
        # plt.imshow(cv2.cvtColor(aug_image, cv2.COLOR_BGR2RGB))
        # plt.axis('off')
        # plt.show()
    copy_image_path1 = copy_image_path.split('/')[-1]
    #polygon_image = copy_image_path1.replace('2025', '')
    save_path_jpg = copy_image_path #polygon_image.replace('.jpg', '_ag.jpg')  # 저장할 파일명 (ex: 0001.jpg → 0001_augmented.jpg)
    cv2.imwrite(save_path + copy_image_path1, aug_image)
    #os.makedirs(save_path, exist_ok=True)

    print(f"Augmented image saved to {save_path_jpg}")



In [ ]:
copy_image_path

In [ ]:
import os 
# 파일 경로
image_path = './DC/dc1.jpg'
xml_path = r'F:\2.프로젝트\파나시아\2025\model3\DC\dc1.xml'
copy_image_path = 'F:/2.프로젝트/파나시아/2025/model3/dataset/dc 추가하기/4/'

for i in os.listdir(copy_image_path)[1:]:
    save_augmented_image(image_path, copy_image_path+ i , xml_path , save_path='./dataset/dc 추가하기/4/aug/')

### **클래스별 객체 분포 파악**

In [1]:
import os
from collections import Counter

def count_yolo_classes(label_dir):
    class_counter = Counter()
    
    for filename in os.listdir(label_dir):
        if filename.endswith('.txt'):
            label_path = os.path.join(label_dir, filename)
            with open(label_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.strip() == '':
                        continue
                    class_id = int(line.strip().split()[0])
                    class_counter[class_id] += 1
                    
    return class_counter

label_dir = 'F:/2.프로젝트/파나시아/2025/model2/dataset/train/labels'  # 라벨 파일(.txt)들이 있는 폴더 경로
class_counts = count_yolo_classes(label_dir)

# 결과 출력
for class_id, count in sorted(class_counts.items()):
    print(f"Class {class_id}: {count} objects")


Class 0: 1578 objects
Class 1: 988 objects
Class 2: 2001 objects
Class 3: 405 objects
Class 4: 820 objects


In [ ]:
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2

# AugMix에서 사용할 기본 포토메트릭 증강들 (기하학 X)
AUGMIX_OPS = [
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=1.0),
    A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1.0),
    A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=25, p=1.0),
    A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
    A.MotionBlur(blur_limit=7, p=1.0),
]

def augmix_single(image, severity=1, width=3, alpha=1.0):
    """분류용 AugMix 형태를 detection에 맞게 포토메트릭만 사용."""
    ws = np.random.dirichlet([alpha] * width)
    m = np.random.beta(alpha, alpha)

    mix = np.zeros_like(image, dtype=np.float32)

    for i in range(width):
        image_aug = image.copy()
        depth = np.random.randint(1, 4)  # 몇 번 연달아 적용할지

        for _ in range(depth):
            op = random.choice(AUGMIX_OPS)
            image_aug = op(image=image_aug)['image']

        mix += ws[i] * image_aug.astype(np.float32)

    mixed = (1 - m) * image.astype(np.float32) + m * mix
    mixed = np.clip(mixed, 0, 255).astype(np.uint8)
    return mixed

class AugMixDet(A.ImageOnlyTransform):
    def __init__(self, always_apply=False, p=0.5, severity=1, width=3, alpha=1.0):
        super().__init__(always_apply, p)
        self.severity = severity
        self.width = width
        self.alpha = alpha

    def apply(self, img, **params):
        return augmix_single(img, self.severity, self.width, self.alpha)

# 사용 예시:
transform_augmix = A.Compose(
    [
        AugMixDet(p=1.0),   # 여기만 AugMix
    ],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'])
)
